# "You Snooze, You Win" Challenge

Every year, the [PhysioNet/CinC (Computing in Cardiology) Challenge](https://www.physionet.org/challenge/) invites "participants to tackle clinically interesting problems that are either unsolved or not well-solved." As you may have figured out from the title, this year's challenge focuses on sleep, particularly the classification of nonarousal and arousal timeframes. If you would like to understand the biological implications of the challenge, we recommend reading PhysioNet's [introduction](https://physionet.org/challenge/2018/) of the challenge.

For this challenge, you will classify samples into 5 classes (Arousal, NREM1, NREM2, NREM3, REM). Each sample consists of seven physiological signals (O2-M1, E1-M2, Chin1-Chin2, ABD, CHEST, AIRFLOW, ECG) measured at 200 Hz over a 60 second period (12000 timepoints). In this notebook, we provide code to import the data, visualize sample signals, implement an example classifier, and 'score' your model.

### Note:
Depending on the model you plan to use, you may want to grab a GPU!

In [0]:
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from sklearn import metrics
from sklearn.utils import shuffle
import tensorflow as tf
from sklearn.model_selection import train_test_split
tf.enable_eager_execution()

## Loading the Dataset

This dataset is a modified version of the PhysioNet/CinC Challenge data, which were contributed by the Massachusetts General Hospital’s Computational Clinical Neurophysiology Laboratory, and the Clinical Data Animation Laboratory.
***
**Class labels:**
- 0 = Arousal
- 1 = NREM1
- 2 = NREM2
- 3 = NREM3
- 4 = REM
***
**Class description:**

|Class|State|Characterizations|Brain Waves|Percentage of Sleep|
|-|-|-|-|-|-|
|Arousal|Consciousness|Wakefulness (coherent cognitive and behavioral responses to external stimuli)|Alpha, Beta|-|
|NREM1|Light sleep|Hypnic jerk (involuntary twitch that causes an individual to awaken for a moment)|Theta|5|
|NREM2|Unequivocal sleep|Sleep spindle (sudden burst of oscillatory brain activity); K-complex (delta wave that lasts for a second)|Theta, Delta|40-50|
|NREM3|Deep sleep|Parasomnias (sleep disorders such as sleepwalking and night terrors)|Delta|15-25|
|REM|Dreaming sleep|REM atonia (paralysis of nonessential skeletal muscles); Dreaming|Alpha, Beta|20-25|
***
**Physiological signal description:**

O2-M1 - posterior brain activity (electroencephalography)

E1-M2 - left eye activity (electrooculography)

Chin1-Chin2 - chin movement (electromyography)

ABD - abdominal movement (electromyography)

CHEST - chest movement (electromyography)

AIRFLOW - respiratory airflow

ECG - cardiac activity (electrocardiography)
***
Run both cell blocks to get the challenge data.

In [0]:
# this line will remove the datasets folder if you've already cloned it
!rm -rf datasets

# and this will clone it again!
!git clone https://github.com/BeaverWorksMedlytics/datasets.git

In [0]:
os.chdir('./datasets/Week2_Challenge/')

## Data Visualization

Run the first cell once to store the training and test sample file locations. If the data have been imported correctly, the cell should output '4000' (training) and '1000' (test).

Run the second cell once to initialize important functions that you may find useful. Descriptions of input and output will be provided for every function in the notebook.

Run the last cell to visualize a random test sample's seven physiological signals in a raw and FFT (fast fourier transform) format. Note that you can change different parameters, which we will go over in detail.

It is important to recognize that the same signal from different samples in the same class may vary in terms of amplitude and frequency. Of course, this is a byproduct of intraspecies variation. Further data preprocessing and/or discriminatory feature extraction may be necessary to account for this phenomenon.

In [0]:
def get_sample_data(data_type, id_number):
  
  """ get the data frame of a single data point by type (training or test) and id (0-999 for test, 0-3999 for training)"""
  
  file = './' + data_type + '/' + str(id_number) + '.xz'
  sample = pd.read_pickle('./' + file)
  return sample, file.split('/')[2]


def get_raw_signals(sample):
  
  """ get the time (x) and signal (y) for a single data point from the sample dataframe"""
  
  raw_signals_x = np.arange(0, 60, step = 1/200)
  raw_signals_y = np.transpose(sample['Signal'][0])
  return (raw_signals_x, raw_signals_y, 'Raw')

In [0]:
""" Initalize key reference dictionaries """
sig_dict = {0:'O2-M1', 1:'E1-M2', 2:'Chin1-Chin2', 3:'ABD', 4:'CHEST', 5:'AIRFLOW', 6:'ECG'}
sig_type_dict = {0:'Time (s)', 1:'Frequency (Hz)'}
stage_dict = {0:'Arousal', 1:'NREM1', 2:'NREM2', 3:'NREM3', 4:'REM'}

This code can be run for training/validation, as well as the final training and application to test data.
* use_test_bool = False --> run training and validation, ignore test data
* use_test_bool = True --> re-train algorithm on full training data, apply to test data

In [0]:
# set to 'False' to run training and validation
# set to 'True' to train on full training data and apply to test data
use_test_bool = False

## Read in the data

In [0]:
# number of classes we're trying to predict
num_classes = 5

# extract features from these channels (you may choose not to use them all!)
channels = [0,1,2,3,4,5,6]

# y labels of training data are in order
y_train = np.concatenate([np.zeros(shape=(800,1)),np.ones(shape=(800,1)),2*np.ones(shape=(800,1)),3*np.ones(shape=(800,1)),4*np.ones(shape=(800,1))])

# initialize array for holding the raw data
X_train = np.ndarray(shape=(4000,12000,len(channels)))


# load in the training data
for ix_train in range(4000):
    samp, fname = get_sample_data('training',ix_train)
    t, sig, _ = get_raw_signals(samp)
    
    X_train[ix_train,:,:] = sig[channels,:].transpose()
    
    # print out our progress so far...
    if ix_train%500 == 0:
      print("{} of 4000 training data loaded".format(ix_train))
    
    
# if we're using the test data, load that in here
if(use_test_bool):  
  X_test = np.ndarray(shape=(1000,12000,len(channels)))
  for ix_test in range(1000):
      samp, fname = get_sample_data('test',ix_test)
      t, sig, type = get_raw_signals(samp)

      X_test[ix_test] = sig.transpose()  

      if ix_test%200 == 0:
        print("{} of 1000 testing data loaded".format(ix_test))

print('Done.')

In [0]:
# Now that we've read in our data, remove the raw data to free up space
!rm -rf datasets

In [0]:
# convert y labels into one-hot vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes)


# if we're not using the test data, split our training data into train/validation sets
if not use_test_bool:
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

## Pre-process our input features
Here, we will do a few things:

1. Subsample our data so it takes up less memory (we will lose some information)
2. Standardize the signals to a unit normal distribution
3. Calculate the Fourier transform

After these procedures our new input features will be in the frequency domain, not the time domain.  

If you want to keep your input in the time domain, simply skip the FFT step.

In [0]:
def standardize_data(Xtrain, Xtest=None):
  """
  Standardize our features to unit-normal distribution.
  Make sure you standardize using values calculated from Xtrain only!!!
  """
  
  # number of channels (we will standardize each separately)
  nchannels = Xtrain.shape[2]
  
  # initialize arrays
  Xavg = np.zeros(shape=(nchannels,1))
  Xstd = np.zeros(shape=(nchannels,1))
  
  for channel in range(nchannels):
    
    # calculate the mean and standard deviation of all the TEST data
    Xavg[channel] = X_train[:,:,channel].mean()
    Xstd[channel] = X_train[:,:,channel].std()
    
    
    # subtract the mean, divide by standard deviation
    Xtrain[:,:,channel] = (X_train[:,:,channel]-Xavg[channel])/Xstd[channel]
    
    # Make sure we are standardizing based on the mean and std of the training data!
    if Xtest is not None:
      Xtest[:,:,channel] = (Xtest[:,:,channel]-Xavg[channel])/Xstd[channel]
      

  return Xtrain, Xtest
  

In [0]:
def subsample_data(X,k=2):
  """ Sample every k'th data point of the data """
  
  return X[:,range(0,X.shape[1],k),:]

In [0]:
def get_FFT(X):
  """ Calculate the Fourier transform of each data point of X"""
  
  npoints = X.shape[0]
  N = X.shape[1]
  nchannels = X.shape[2]
  
  # initialize array
  X2 =np.zeros(shape=(X.shape[0],X.shape[1]//2,X.shape[2]))
  
  # for each channel ...
  for channel in range(nchannels):
    
    # for each sample ...
    for pt in range(npoints):
      
      # calculate the Fourier transform
      ft = np.fft.fft(X[pt,:,channel])  
      
      # only store the first 1/2 of the Fourier transform array
      X2[pt,:,channel] = np.abs(ft)[:N // 2] * 2 / N
    
  return X2
  

In [0]:
# Subsample the data to reduce the size of our inputs
# NOTE: we may lose some of the very high-frequency information, but we may decide that's okay.

k=5

X_train = subsample_data(X_train,k=k)

if use_test_bool:  X_test = subsample_data(X_test,k=k)
else:              X_val  = subsample_data(X_val,k=k)

# Note that our original data was shape (4000, 12000, n_channels) -- by subsampling we've reduced the size of our inputs
X_train.shape

In [0]:
# Standardize the data
if(use_test_bool): 
  X_train, X_test = standardize_data(X_train, X_test)
  
else:
  X_train, X_val = standardize_data(X_train, X_val)



In [0]:
# Get the FFT of our subsampled, standardized data
X_train = get_FFT(X_train)
if(use_test_bool): 
  X_test = get_FFT(X_test)
else:
  X_val = get_FFT(X_val)


In [0]:
# After taking the FFT, the size of our data should be cut in half
X_train.shape

## Create our model
Here are a few different TensorFlow.Keras models.  Feel free to experiment with different models, some will work better than others for different tasks.  Most of these use 1-D Convolution layers which is particularly appropriate in the time domain.

In [0]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, GlobalMaxPooling1D


def make_model(input_shape, modelnum=1):
  
  # All models are sequential
  model = tf.keras.Sequential()
  
  if modelnum == 1:
    """ Brian's Model (Almost)"""
    
    model.add(Conv1D(16, kernel_size=5, activation=tf.nn.relu, input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=3, padding='valid'))
    model.add(Dropout(0.1))    
    model.add(Conv1D(16, kernel_size=5, activation=tf.nn.relu))
    model.add(MaxPooling1D(pool_size=3, padding='valid'))
    model.add(Dropout(0.1))
    model.add(Flatten())


  elif modelnum == 2:
    """ Two convolutional layers with dropout, should be pretty fast to train """
    
    model.add(Conv1D(32, kernel_size=5, padding="same", activation=tf.nn.relu, input_shape=input_shape)) 
    model.add(Dropout(0.1))
    model.add(Conv1D(32, kernel_size=5, padding="same", activation=tf.nn.relu)) 
    model.add(Dropout(0.1))
    model.add(GlobalMaxPooling1D()) 
    
  elif modelnum == 3:
    """ This model is similar to Model #2 but with different kernel sizes"""
    
    model.add(Conv1D(32, kernel_size=15, activation=tf.nn.relu, input_shape=input_shape)) 
    model.add(Dropout(0.1))
    model.add(Conv1D(32, kernel_size=15, activation=tf.nn.relu)) 
    model.add(Dropout(0.1))
    model.add(GlobalMaxPooling1D()) 
    
  elif modelnum == 4:
    """ Also similar to Model #2"""
    
    model.add(Conv1D(64, kernel_size=9, activation=tf.nn.relu, input_shape=input_shape)) 
    model.add(Dropout(0.2))
    model.add(Conv1D(32, kernel_size=9, activation=tf.nn.relu)) 
    model.add(Dropout(0.2))
    model.add(GlobalMaxPooling1D()) 
    
    
  elif modelnum == 5:
    """ Single convolution layer model"""
    
    model.add(Conv1D(filters=64, kernel_size=20, strides=1, padding="same",  activation = tf.nn.relu, input_shape=input_shape))
    model.add(GlobalMaxPooling1D())
    
  elif modelnum == 6:
    """ This model is similar to Model #4 but with slightly different parameters"""
    
    model.add(Conv1D(32, kernel_size=5, padding="valid", activation=tf.nn.relu, input_shape=input_shape)) 
    model.add(Dropout(0.1))
    model.add(Conv1D(32, kernel_size=5, padding="valid", activation=tf.nn.relu)) 
    model.add(Dropout(0.1))
    model.add(GlobalMaxPooling1D()) 

  elif modelnum == 7:
    """ This model doesn't use convolutions at all!"""
    
    model.add(MaxPooling1D(pool_size=15, padding='valid', input_shape=input_shape))
    model.add(Dropout(0.1))
    model.add(Flatten())
    model.add(Dense(10, activation=tf.nn.relu))


  else:
    return None
  
  # All models end with a dense layer with softmax activation
  model.add(Dense(num_classes, activation=tf.nn.softmax))
  return model

In [0]:
# Choose which model you want to train

model = make_model(input_shape=X_train[0].shape, modelnum=1)
model.summary()

## Compile and train the model

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.train.AdamOptimizer(0.001),
              metrics=['accuracy'])

In [0]:
batch_size=200
epochs=100

if use_test_bool:
  history = model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=2)
else:
  history = model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(X_val, y_val),
              verbose=2)

## Plot the Training (and Validation) Accuracy over training epochs

In [0]:
plt.figure()
plt.plot(history.history['acc'],label='training')
if not use_test_bool:
  plt.plot(history.history['val_acc'],label='validation')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.figure()
plt.plot(history.history['loss'],label='training')
if not use_test_bool:
  plt.plot(history.history['val_loss'],label='validation')
plt.xlabel('Epochs')
plt.ylabel('Loss (Cross-Entropy)')
plt.legend()
plt.show()

## Evaluate the Model

- **Area Under the ROC Curve (AUCROC)**: The receiver operating characteristic (ROC) curve plots the true positive rate (sensitivity/recall) against the false positive rate (fall-out) at many decision threshold settings. The area under the curve (AUC) measures discrimination, the classifier's ability to correctly identify samples from the "positive" and "negative" cases. Intuitively, AUC is the probability that a randomly chosen "positive" sample will be labeled as "more positive" than a randomly chosen "negative" sample. In the case of a multi-class ROC curve, each class is considered separately before taking the weighted average of all the class results. Simply put, the class under consideration is labeled as "positive" while all other classes are labeled as "negative." Below is the multi-class ROC curve for the example classifier. The AUCROC score should be between 0 and 1, in which 0.5 is random classification and 1 is perfect classification.

 ![alt text](https://image.ibb.co/g4pzST/ROCAUC.png)

- **Matthews Correlation Coefficient (MCC)**: The MCC measures the quality of binary classifications, irrespective of the class sizes. Importantly, it is typically regarded as a balanced measure since it considers all values in the 2x2 contingency table (TP, FP, TN, FN). For this challenge, the binary classes will be "Arousal" (Arousal) and "Nonarousal" (NREM1, NREM2, NREM3, REM). The MCC score should be between -1 and 1, in which 0 is random classification and 1 is perfect classification.

 ![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/5caa90fc15105b74b59a30bbc9cc2e5bd43a13b7)

Using these metrics, the example classifier has the following scores on test data:
- AUCROC: 0.755
- MCC: 0.353
- Creativity: ( ͡° ͜ʖ ͡°)


Below is the code used to calculate the AUCROC and MCC metrics when evaluating your classifier (shown evaluated on the validation data)

In [0]:
# ROC AUC on validation data

if not use_test_bool:
  
  y_val_pred = model.predict(X_val)
  y_val_pred = pd.DataFrame(y_val_pred)
  y_val = pd.DataFrame(y_val)
  
  fpr = {}
  tpr = {}
  roc_auc = {}
  
  plt.figure(figsize=(8,8))
  for i in range(num_classes):
      fpr[i], tpr[i], _ = metrics.roc_curve(y_val.iloc[:, i], y_val_pred.iloc[:, i])
      roc_auc[i] = metrics.auc(fpr[i], tpr[i])
      plt.plot(fpr[i], tpr[i], label = stage_dict[i] + ', ' + str(i), lw=3)   
  
  plt.plot([0, 1], [0, 1],':')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Multi-Class ROC Curve')
  plt.legend(fontsize='large')
  plt.show()

  fpr["micro"], tpr["micro"], _ = metrics.roc_curve(y_val.values.ravel(), y_val_pred.values.ravel())
  roc_auc = metrics.auc(fpr["micro"], tpr["micro"])
  print('ROC AUC: {:0.3f}'.format(roc_auc))

In [0]:
# MCC on validation data

if not use_test_bool:
  y_true = []
  y_pred = []

  test_true = y_val.idxmax(axis=1)
  test_predict = y_val_pred.idxmax(axis=1)

  for i in range(y_val_pred.shape[0]):

      if test_predict.iloc[i]==0: y_pred.append(1)
      else: y_pred.append(-1)

      if test_true[i]==0: y_true.append(1)
      else: y_true.append(-1)

  mcc = metrics.matthews_corrcoef(y_true, y_pred)
  print('MCC: {:0.3f}'.format(mcc))


## If you're applying to test data, use the model you've just trained to predict test labels

In [0]:
if use_test_bool:

  y_test_pred = model.predict(X_test)
  y_test_pred = pd.DataFrame(y_test_pred)
  
  # Take a peek, make sure the output makes sense
  print(y_test_pred.head(10))

## Write predictions to file

In [0]:
filename = 'My_Test_Predictions.xz'
y_test_pred.to_pickle(filename)

To download the pickle file to your computer, this line should work:

In [0]:
files.download(filename)

If it doesn't work (it doesn't work for me...) you will need to give Colab permission to write the file to your Google Drive, and then you can retrieve the file from there.

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Save the pickle file to your Google Drive
file = drive.CreateFile()
file.SetContentFile(filename)
file.Upload() 

## Save your TensorFlow model
I had trouble getting Colab to cooperate, but perhaps you will have better luck!

In [0]:
tf.keras.models.save_model(
    model,'my_tf_model')